# Results Analysis

Analyze backtest results, calculate metrics, and generate visualizations.

**Version**: v1.11.0  
**Architecture**: Modular (lib/metrics/, lib/plots/, lib/paths/, lib/validation/)

## Configuration

Set the strategy name and result directory below.

**Note**: This notebook uses the v1.11.0 modular architecture. All imports use canonical paths from `lib/` packages.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'


In [ ]:
# Setup
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Standard library imports
import json

# Third-party imports
import pandas as pd
import numpy as np
from IPython.display import Image, display

# Local imports - lib modules (v1.11.0 modular architecture)
# Metrics and plots modules provide comprehensive analysis capabilities
from lib.paths import get_project_root, get_results_dir
from lib.metrics import (
    calculate_metrics,
    calculate_trade_metrics,
    calculate_rolling_metrics
)
from lib.plots import (
    plot_all,
    plot_equity_curve,
    plot_drawdown,
    plot_monthly_returns,
    plot_trade_analysis
)


In [ ]:
# List available results (optional)
results_base = get_results_dir() / strategy_name
if results_base.exists():
    result_dirs = sorted([d for d in results_base.iterdir() if d.is_dir() and not d.name == 'latest'])
    print(f"Available results for '{strategy_name}':")
    for i, d in enumerate(result_dirs[-5:], 1):  # Show last 5
        print(f"  {i}. {d.name}")
    print(f"\nUsing latest: {results_base / 'latest'}")
else:
    print(f"⚠ No results found for strategy '{strategy_name}'")
    print(f"  Run backtest first: python scripts/run_backtest.py --strategy {strategy_name}")

In [ ]:
# Validate Results Data
from lib.validation import BacktestValidator

print("\nValidating results data...")
try:
    validator = BacktestValidator()
    
    # Validate returns
    if 'returns' in perf.columns:
        returns_validation = validator.validate_returns(perf['returns'])
        if returns_validation.is_valid:
            print("  ✓ Returns data valid")
        else:
            print("  ⚠ Returns data validation issues:")
            for issue in returns_validation.issues[:3]:
                print(f"    - {issue.message}")
    
    # Validate transactions if available
    if transactions is not None and len(transactions) > 0:
        transactions_validation = validator.validate_transactions(transactions)
        if transactions_validation.is_valid:
            print("  ✓ Transactions data valid")
        else:
            print("  ⚠ Transactions data validation issues:")
            for issue in transactions_validation.issues[:3]:
                print(f"    - {issue.message}")
    
except Exception as e:
    print(f"  ⚠ Validation error: {e}")

In [ ]:
# Load results
result_dir = None  # None = use latest, or specify: Path('results/strategy/backtest_20240101_120000')

# Resolve result directory
if result_dir is None:
    results_base = get_results_dir() / strategy_name / 'latest'
    if results_base.is_symlink():
        result_dir = results_base.resolve()
    else:
        result_dir = results_base
    print(f"Using latest results: {result_dir}")
else:
    result_dir = Path(result_dir)
    print(f"Using specified results: {result_dir}")

# Verify results directory exists
if not result_dir.exists():
    raise FileNotFoundError(
        f"Results directory not found: {result_dir}\n"
        f"  Run backtest first: python scripts/run_backtest.py --strategy {strategy_name}"
    )

# Load performance data
try:
    returns_file = result_dir / 'returns.csv'
    positions_file = result_dir / 'positions.csv'
    transactions_file = result_dir / 'transactions.csv'
    metrics_file = result_dir / 'metrics.json'
    
    perf = pd.read_csv(returns_file, index_col=0, parse_dates=True)
    positions = pd.read_csv(positions_file, index_col=0, parse_dates=True) if positions_file.exists() else None
    transactions = pd.read_csv(transactions_file, index_col=0, parse_dates=True) if transactions_file.exists() else None
    
    with open(metrics_file) as f:
        metrics = json.load(f)
    
    print(f"✓ Loaded results from: {result_dir}")
    print(f"  Returns: {len(perf)} days")
    print(f"  Positions: {len(positions) if positions is not None else 0} records")
    print(f"  Transactions: {len(transactions) if transactions is not None else 0} records")
    
except Exception as e:
    print(f"✗ Error loading results: {e}")
    raise


In [ ]:
# Validate Results Data
from lib.validation import BacktestValidator

print("\nValidating results data...")
try:
    validator = BacktestValidator()
    
    # Validate returns
    if 'returns' in perf.columns:
        returns_validation = validator.validate_returns(perf['returns'])
        if returns_validation.is_valid:
            print("  ✓ Returns data valid")
        else:
            print("  ⚠ Returns data validation issues:")
            for issue in returns_validation.issues[:3]:
                print(f"    - {issue.message}")
    
    # Validate transactions if available
    if transactions is not None and len(transactions) > 0:
        transactions_validation = validator.validate_transactions(transactions)
        if transactions_validation.is_valid:
            print("  ✓ Transactions data valid")
        else:
            print("  ⚠ Transactions data validation issues:")
            for issue in transactions_validation.issues[:3]:
                print(f"    - {issue.message}")
    
except Exception as e:
    print(f"  ⚠ Validation error: {e}")

# Display metrics summary
print("\n" + "=" * 60)
print("PERFORMANCE METRICS")
print("=" * 60)

# Core metrics
print(f"Total Return: {metrics.get('total_return', 0):.2%}")
print(f"Annual Return: {metrics.get('annual_return', 0):.2%}")
print(f"Sharpe Ratio: {metrics.get('sharpe', 0):.3f}")
print(f"Sortino Ratio: {metrics.get('sortino', 0):.3f}")
print(f"Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
print(f"Calmar Ratio: {metrics.get('calmar', 0):.3f}")
print(f"Annual Volatility: {metrics.get('annual_volatility', 0):.2%}")

# Trade metrics (if available)
if 'trade_count' in metrics and metrics['trade_count'] > 0:
    print("\nTrade Metrics:")
    print(f"  Trade Count: {metrics.get('trade_count', 0)}")
    print(f"  Win Rate: {metrics.get('win_rate', 0):.2%}")
    print(f"  Profit Factor: {metrics.get('profit_factor', 0):.3f}")
    print(f"  Avg Win: {metrics.get('avg_win', 0):.2%}")
    print(f"  Avg Loss: {metrics.get('avg_loss', 0):.2%}")

print("=" * 60)

# Calculate additional metrics if needed
if 'returns' in perf.columns:
    returns = perf['returns']
    additional_metrics = calculate_metrics(returns)
    print("\nAdditional Metrics (calculated):")
    print(f"  Skewness: {additional_metrics.get('skewness', 0):.3f}")
    print(f"  Kurtosis: {additional_metrics.get('kurtosis', 0):.3f}")


In [ ]:
# Calculate rolling metrics (optional)
if 'returns' in perf.columns:
    returns = perf['returns']
    
    try:
        rolling_metrics = calculate_rolling_metrics(
            returns,
            window=30,  # 30-day rolling window
            metrics=['sharpe', 'sortino', 'volatility']
        )
        
        print("\nRolling Metrics (30-day window):")
        print(rolling_metrics.tail(10))  # Show last 10 periods
        
    except Exception as e:
        print(f"⚠ Rolling metrics calculation error: {e}")

In [ ]:
# Generate plots
try:
    # Option 1: Generate all plots at once
    plot_all(
        perf=perf,
        save_dir=result_dir,
        show=False  # Set to True to display inline
    )
    print("✓ Generated all plots")
    
    # Option 2: Generate individual plots (uncomment to use)
    # plot_equity_curve(perf, save_path=result_dir / 'equity_curve_custom.png')
    # plot_drawdown(perf, save_path=result_dir / 'drawdown_custom.png')
    # plot_monthly_returns(perf, save_path=result_dir / 'monthly_returns_custom.png')
    
    # Display existing plots
    plot_files = [
        'equity_curve.png',
        'drawdown.png',
        'monthly_returns.png',
        'rolling_metrics.png',
        'trade_analysis.png',
    ]
    
    for plot_file in plot_files:
        plot_path = result_dir / plot_file
        if plot_path.exists():
            print(f"\n{plot_file.replace('.png', '').replace('_', ' ').title()}:")
            display(Image(str(plot_path)))
    
except Exception as e:
    print(f"⚠ Plot generation error: {e}")
    print(f"  Some plots may not be available")


In [ ]:
# Trade analysis
if transactions is not None and len(transactions) > 0:
    try:
        trade_metrics = calculate_trade_metrics(transactions, perf)
        
        print("\n" + "=" * 60)
        print("TRADE ANALYSIS")
        print("=" * 60)
        print(f"Total Trades: {trade_metrics.get('trade_count', 0)}")
        print(f"Winning Trades: {trade_metrics.get('winning_trades', 0)}")
        print(f"Losing Trades: {trade_metrics.get('losing_trades', 0)}")
        print(f"Win Rate: {trade_metrics.get('win_rate', 0):.2%}")
        print(f"Average Win: ${trade_metrics.get('avg_win', 0):.2f}")
        print(f"Average Loss: ${trade_metrics.get('avg_loss', 0):.2f}")
        print(f"Profit Factor: {trade_metrics.get('profit_factor', 0):.3f}")
        print("=" * 60)
        
    except Exception as e:
        print(f"⚠ Trade analysis error: {e}")
else:
    print("ℹ No transaction data available for trade analysis")